### Delete the file with the name with which we will create a new file

In [18]:
from os import remove
from os.path import isfile

xlsx = 'unite.xlsx'

if isfile(xlsx): remove(xlsx)

from glob import glob
fxls = glob('*.xlsx')
fxls

['Book1.xlsx', 'Book3.xlsx', 'Book2.xlsx']

### Сreate a file with this name

In [19]:
from pandas import ExcelWriter
writer = ExcelWriter(xlsx, engine='xlsxwriter')
writer.save()

### The function of transferring pages from one file to a new one

In [20]:
def save(this, sheet):
    writer = ExcelWriter(xlsx, mode="a", engine="openpyxl")
    this.to_excel(writer, sheet_name=f"{sheet}", index=False)
    writer.save()

### Iterate over files and pages of files to save their contents to a new file

In [21]:
from pandas import ExcelFile, read_excel
for fxl in fxls:
    ef = ExcelFile(fxl)
    for sheet in ef.sheet_names:
        save(this=read_excel(ef, sheet_name=sheet), sheet=sheet)
        print(fxl, ":", sheet)

Book1.xlsx : FIRST
Book1.xlsx : SECOND
Book1.xlsx : THIRD
Book3.xlsx : FIRST3
Book3.xlsx : SECOND3
Book3.xlsx : THIRD3
Book2.xlsx : FIRST2
Book2.xlsx : SECOND2
Book2.xlsx : THIRD2


#### When we create a file, a page with a known name is also created, we can rename this page

In [22]:
from openpyxl import load_workbook

def rename():
    fxl = load_workbook(xlsx)
    fxl.get_sheet_by_name('Sheet1').title = 'result'
    fxl.save(xlsx)

rename()

/var/folders/w8/g98rrphn3bx_tzc3v6hvjn5r0000gn/T/ipykernel_76956/4077244973.py:5: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  fxl.get_sheet_by_name('Sheet1').title = 'result'


### Now we can find out what pages UR file consists of

In [23]:
fxl = load_workbook(xlsx)
sheets = fxl.sheetnames
print(sheets)

['result', 'FIRST', 'SECOND', 'THIRD', 'FIRST3', 'SECOND3', 'THIRD3', 'FIRST2', 'SECOND2', 'THIRD2']


### Add data to empty page

In [24]:
def fill(xlsx, sheet, where, what):
    fxl = load_workbook(f'{xlsx}')
    sheet = fxl.get_sheet_by_name(f'{sheet}')
    sheet[where]= str(what)
    fxl.save(f'{xlsx}')

sheet = 'result'
where = 'A1'
what = 'HOWDY GUEST'
fill(xlsx, sheet, where, what)

/var/folders/w8/g98rrphn3bx_tzc3v6hvjn5r0000gn/T/ipykernel_76956/177910856.py:3: DeprecationWarning: Call to deprecated function get_sheet_by_name (Use wb[sheetname]).
  sheet = fxl.get_sheet_by_name(f'{sheet}')


### Read information from a cell

In [8]:
fxl = load_workbook(f'{xlsx}',read_only=False, keep_vba= True)
cell_obj = fxl['result'].cell(row=1, column=1)
print(f'[A1]= {cell_obj.value}')


[A1]= HOWDY GUEST


### U can also use Workbook to easily specify the address for the record

In [9]:
from openpyxl import Workbook

wb = Workbook()
ws = wb.active
ws['A1'] = 0
ws.append([1, 2, 3])
# wb.save("now_file.xlsx")

### To see the result in the file run this...

In [13]:
from os import system
system('open unite.xlsx')

0